## Installation of Object Detection API

for /f %i in ('dir /b object_detection\protos\*.proto') do protoc object_detection\protos\%i --python_out=.

^ Use this command instead of the normal protoc one

In [1]:
import os
import shutil
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [2]:
os.listdir()

['.ipynb_checkpoints',
 'instructions.txt',
 'locateMouse.py',
 'main.py',
 'processImg.py',
 'TestDetection.ipynb',
 'testDetection.py',
 'Train.ipynb',
 'Workspace']

### Require Changes

Place all images and annotations in Workspace/images/train OR Workspace/images/test


Imgs and annots are in the same folder

In [3]:
# Folder containing tensorflow files (Installed via github)
TENSORFLOW_PATH = "../RealTimeObjectDetection/Tensorflow"

# Folder containing all own files (Images, Annotations, Models, etc..)
WORKSPACE_PATH = "Workspace"

# Tensorflow script to generate tf Record
SCRIPTS_PATH = TENSORFLOW_PATH + "/scripts"
APIMODEL_PATH = TENSORFLOW_PATH + "/models"

ANNOTATION_PATH = WORKSPACE_PATH + "/annotations"
IMAGE_PATH = WORKSPACE_PATH + '/images'
MODEL_PATH = WORKSPACE_PATH + '/my_models'
PRETRAINED_MODEL_PATH = WORKSPACE_PATH + "/pre-trained-models"

# Change this
CUSTOM_MODEL_NAME = 'over_ssdnet'
CONFIG_PATH = MODEL_PATH + f'/{CUSTOM_MODEL_NAME}/pipeline.config'
CHECKPOINT_PATH = MODEL_PATH + f'/{CUSTOM_MODEL_NAME}/'

### Require Changes

In [4]:
# Change this
labels = [{'name': 'head', 'id': 1}]

with open(ANNOTATION_PATH + '/label_map.pbtxt', 'w+') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

In [5]:
os.listdir(TENSORFLOW_PATH)

['models', 'scripts', 'workspace']

In [6]:
!python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x {IMAGE_PATH + '/train'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/train.record'}
!python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x{IMAGE_PATH + '/test'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/test.record'}

Successfully created the TFRecord file: Workspace/annotations/train.record
Successfully created the TFRecord file: Workspace/annotations/test.record


In [7]:
# Copy the pretrained pipeline.config file to the \models\CUSTOM_MODEL_NAME folder

OVERWATCH_MODEL_PATH = MODEL_PATH + '/' + CUSTOM_MODEL_NAME
if not os.path.exists(OVERWATCH_MODEL_PATH):
    os.mkdir(OVERWATCH_MODEL_PATH)

pretrained_config_path = PRETRAINED_MODEL_PATH + '/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config'
shutil.copy(pretrained_config_path, OVERWATCH_MODEL_PATH)

'Workspace/my_models/over_ssdnet\\pipeline.config'

In [8]:
CONFIG_PATH = OVERWATCH_MODEL_PATH + '/pipeline.config'
config = config_util.get_configs_from_pipeline_file(CONFIG_PATH)

In [9]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(CONFIG_PATH, "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()
    # Pass in the config string as first param, and the config variable as second param
    text_format.Merge(proto_str, pipeline_config)

In [10]:
ANNOTATION_PATH

'Workspace/annotations'

In [11]:
# 2 classes
pipeline_config.model.ssd.num_classes = 1
# Set batch size to 4
pipeline_config.train_config.batch_size = 4
# Where you want your model to start training from. Here, you want to utilize the checkpoint-0 in the pre-trained model (Transfer Learning)
pipeline_config.train_config.fine_tune_checkpoint = PRETRAINED_MODEL_PATH + '/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0'
# Specify the type of task
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
# Specify your label_map_path and the tfrecords for train & test
pipeline_config.train_input_reader.label_map_path= ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/train.record']
pipeline_config.eval_input_reader[0].label_map_path = ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/test.record']

In [12]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(CONFIG_PATH, "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)

In [13]:
'''
Here, you specify the:
- object_detection_main: Filepath to the object_detection python file in the git repo (tensorflow models) that was cloned
- model_dir: Filepath where you want to save your model checkpoints to
- pipeline_config_path: Filepath to pipeline.config file (This contains the filepaths to tfrecords, labels, batch_size, etc..)
- num_train_steps: Number of epochs
'''

TRAIN_STEPS=50
OBJECT_DETECTION_MAIN = f"{APIMODEL_PATH}/research/object_detection/model_main_tf2.py"
PIPELINE_CONFIG_PATH = OVERWATCH_MODEL_PATH + "/pipeline.config"

In [14]:
print(f"python {OBJECT_DETECTION_MAIN} --model_dir={OVERWATCH_MODEL_PATH} --pipeline_config_path={PIPELINE_CONFIG_PATH} --num_train_steps={TRAIN_STEPS}")

python ../RealTimeObjectDetection/Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=Workspace/my_models/over_ssdnet --pipeline_config_path=Workspace/my_models/over_ssdnet/pipeline.config --num_train_steps=50


In [15]:
# C:\Users\alexc\PycharmProjects\ObjectDetection\Overwatch